# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bargarh,21.3333,83.6167,26.54,29,0,1.99,IN,1708230240
1,1,cabo san lucas,22.8909,-109.9124,20.98,64,0,3.02,MX,1708229916
2,2,vernon,50.2581,-119.2691,-2.06,75,79,1.03,CA,1708230241
3,3,kodiak,57.7900,-152.4072,3.73,100,100,3.60,US,1708230241
4,4,margaret river,-33.9500,115.0667,29.89,44,94,1.64,AU,1708230242


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng",
                                     "Lat",
                                     geo=True,
                                     tiles="OSM",
                                     alpha = 0.75,
                                     frame_width=700,
                                     frame_height=500,
                                     size="Humidity",
                                     color="City",
                                     xlim=(-180,180),
                                     ylim=(-60,75)
                                     )
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df["Max Temp"] <27) & 
                                  (city_data_df["Max Temp"]>21) &
                                  (city_data_df["Humidity"]<45)]


# Drop any rows with null values
filtered_ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
filtered_ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bargarh,21.3333,83.6167,26.54,29,0,1.99,IN,1708230240
5,5,mangrol,21.1167,70.1167,25.33,41,3,3.47,IN,1708230242
10,10,husainabad,24.5333,84.0167,22.26,25,0,2.02,IN,1708230244
23,23,blackmans bay,-43.0167,147.3167,25.29,39,13,3.85,AU,1708230251
109,109,djenne,13.9061,-4.5533,23.26,10,4,3.49,ML,1708230297
161,161,jianshui,23.6363,102.8264,22.85,18,0,6.22,CN,1708230320
237,237,arroyo seco,-33.1549,-60.5086,22.40,10,9,2.08,AR,1708230360
259,259,tura,25.5198,90.2201,24.15,24,3,2.09,IN,1708230370
275,275,athni,16.7333,75.0667,26.65,29,10,4.74,IN,1708230378
369,369,aleg,17.0481,-13.9209,25.90,11,32,4.73,MR,1708230426


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_ideal_city_data_df.loc[:,['City', 'Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name","",True)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bargarh,IN,21.3333,83.6167,29,
5,mangrol,IN,21.1167,70.1167,41,
10,husainabad,IN,24.5333,84.0167,25,
23,blackmans bay,AU,-43.0167,147.3167,39,
109,djenne,ML,13.9061,-4.5533,10,
161,jianshui,CN,23.6363,102.8264,18,
237,arroyo seco,AR,-33.1549,-60.5086,10,
259,tura,IN,25.5198,90.2201,24,
275,athni,IN,16.7333,75.0667,29,
369,aleg,MR,17.0481,-13.9209,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
params = {
    "radius":radius,
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bargarh - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel found
husainabad - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
djenne - nearest hotel: Hotel Chez Baba
jianshui - nearest hotel: 阳阳大酒店
arroyo seco - nearest hotel: Don enrique
tura - nearest hotel: No hotel found
athni - nearest hotel: No hotel found
aleg - nearest hotel: Hotel Oasis Aleg
mount gambier - nearest hotel: The Old Mount Gambier Gaol
kita - nearest hotel: Hotel Chats Rouges
pratapgarh - nearest hotel: No hotel found
am timan - nearest hotel: Nimla
diu - nearest hotel: Hotel Ankur
new norfolk - nearest hotel: Woodbridge on the Derwent
sotouboua - nearest hotel: Verger
sao vicente - nearest hotel: Pousada Vitória
shaoguan - nearest hotel: Holiday Inn Shaoguan Downtown
batangafo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,bargarh,IN,21.3333,83.6167,29,No hotel found
5,mangrol,IN,21.1167,70.1167,41,No hotel found
10,husainabad,IN,24.5333,84.0167,25,No hotel found
23,blackmans bay,AU,-43.0167,147.3167,39,Villa Howden
109,djenne,ML,13.9061,-4.5533,10,Hotel Chez Baba
161,jianshui,CN,23.6363,102.8264,18,阳阳大酒店
237,arroyo seco,AR,-33.1549,-60.5086,10,Don enrique
259,tura,IN,25.5198,90.2201,24,No hotel found
275,athni,IN,16.7333,75.0667,29,No hotel found
369,aleg,MR,17.0481,-13.9209,11,Hotel Oasis Aleg


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng",
                                   "Lat",
                                   geo=True,
                                   tiles="OSM",
                                   alpha = 0.75,
                                   frame_width=700,
                                   frame_height=500,
                                   hover_cols=["Hotel Name","Country"],
                                   color="City"
                                   )

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)